In [ ]:
!pip install transformers

In [3]:
# Importing the relevant modules from transformers import BertTokenizer, BertModel
import csv
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as f

# Load dataset

In [37]:
dev_df = pd.read_csv("dev_sent_emo.csv")
dev_df

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,"Oh my God, hes lost it. Hes totally lost it.",Phoebe,sadness,negative,0,0,4,7,"00:20:57,256","00:21:00,049"
1,2,What?,Monica,surprise,negative,0,1,4,7,"00:21:01,927","00:21:03,261"
2,3,"Or! Or, we could go to the bank, close our acc...",Ross,neutral,neutral,1,0,4,4,"00:12:24,660","00:12:30,915"
3,4,Youre a genius!,Chandler,joy,positive,1,1,4,4,"00:12:32,334","00:12:33,960"
4,5,"Aww, man, now we wont be bank buddies!",Joey,sadness,negative,1,2,4,4,"00:12:34,211","00:12:37,505"
...,...,...,...,...,...,...,...,...,...,...,...
1104,1174,No.,Monica,sadness,negative,113,9,6,2,"00:19:28,792","00:19:29,876"
1105,1175,What? Oh my God! Im gonna miss you so much!,Rachel,sadness,negative,113,10,6,2,"00:19:33,213","00:19:35,965"
1106,1176,Im gonna miss you!,Monica,sadness,negative,113,11,6,2,"00:19:36,175","00:19:37,967"
1107,1177,I mean its the end of an era!,Rachel,sadness,negative,113,12,6,2,"00:19:39,094","00:19:40,928"


In [40]:
df2 = dev_df.drop(dev_df.columns[[0,2,4,5,6,7,8,9,10]],axis=1)
df2

,Utterance,Emotion
0,"Oh my God, hes lost it. Hes totally lost it.",sadness
1,What?,surprise
2,"Or! Or, we could go to the bank, close our acc...",neutral
3,Youre a genius!,joy
4,"Aww, man, now we wont be bank buddies!",sadness
...,...,...
1104,No.,sadness
1105,What? Oh my God! Im gonna miss you so much!,sadness
1106,Im gonna miss you!,sadness
1107,I mean its the end of an era!,sadness


In [41]:
from sklearn.model_selection import train_test_split
from collections import Counter

X_train, X_valid, Y_train, Y_valid= train_test_split(df2['Utterance'].tolist(),\
                                                      df2['Emotion'].tolist(),\
                                                      test_size=0.2,\
                                                      stratify = df2['Emotion'].tolist(),\
                                                      random_state=0)

print('Train data len:'+str(len(X_train)))
print('Class distribution'+str(Counter(Y_train)))


print('Valid data len:'+str(len(X_valid)))
print('Class distribution'+ str(Counter(Y_valid)))

Train data len:887
Class distributionCounter({'neutral': 376, 'joy': 130, 'anger': 122, 'surprise': 120, 'sadness': 89, 'fear': 32, 'disgust': 18})
Valid data len:222
Class distributionCounter({'neutral': 94, 'joy': 33, 'anger': 31, 'surprise': 30, 'sadness': 22, 'fear': 8, 'disgust': 4})


In [42]:
valid_dat =list(zip(Y_valid,X_valid))

In [43]:
import torch
from torch.utils.data import DataLoader

In [44]:
dataloader = DataLoader(valid_dat, batch_size=16,
                              shuffle=True)

In [6]:
data = csv.reader(open('dev_sent_emo.csv', 'rt'), delimiter=",")
text =  [row[1] for row in data]
text = text[1:]
print(text[57])

All in good time my love.


In [33]:
class TextEmbedding():
  def __init__(self):
    super(TextEmbedding, self).__init__()
      
    self.bert = BertModel.from_pretrained('bert-base-uncased') # pre-trained bert model
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # tokenizer

  def embedding(self, text):

    # text: the texts to be tokenized
    # padding: max length: pad the texts to the maximum length (so that all outputs have the same length)
    # return_tensors='pt' : return the tensors (not lists)
    encoding = self.tokenizer(text, padding='max_length', max_length=70, return_tensors='pt') 
   
   
    with torch.no_grad():
      embedded = self.bert(**encoding)[0] # put encoded text through bert model

    return embedded


In [34]:
textt = TextEmbedding()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
x = textt.embedding(text[55])
print(x.shape)

torch.Size([1, 70, 768])


In [51]:
for i, (label, text) in enumerate(dataloader):
  print(text)
  emb = textt.embedding(text)
  print(emb)
  print(emb.shape)
  if i == 1:
    break

('Yep! Grab a plate.', 'Oh, Rachel and I have appointments to get our hair cut.', 'Well Rachel\x92s not here!', 'Listen, the number that you gave me 853-5\x85   That is their old number!', 'I know where Minsk is.', 'You\x92re a genius!', 'Why would they take away our keys?', 'Pizza!! We like Pizza!! Get out!!!', 'You don\x92t have a', 'Easy. Easy. You have to go to the hospital. Okay?', 'Done.', 'Hey listen umm, how would you like to get together?', "Uh, so, where did you say you're from again?", 'Thanks for having me over, you guys.', 'Toby! I\x92m not gonna let you cover for him. Anything you say right now will just get me more upset with Chandler!', 'Hey!')
tensor([[[-1.9630e-01,  2.1479e-01, -6.2158e-02,  ..., -3.0095e-01,
           1.4980e-01,  2.9243e-01],
         [ 6.3506e-01,  2.8202e-01,  3.0873e-01,  ...,  1.1894e-01,
           5.0951e-01, -1.4629e-01],
         [-1.4289e-01, -5.5459e-03,  5.3345e-01,  ...,  4.8339e-01,
           1.8504e-01, -2.6553e-01],
         ...,
  